In [ ]:
from dl1_data_handler.reader import DL1DataReader
from ctlearn.data_loader import setup_DL1DataReader #CTLearn > v0.5.0
import yaml
import matplotlib.pyplot as plt
%matplotlib inline

# directly reader

In [ ]:
reader = DL1DataReader(["/mnt/lromanato/anaconda3/envs/LST37/CTA/smallDataset/MAGIC/GA_za05to35_8_821319_Y_w0_dl1dh_v0_2.9.0.h5"],
                       mode='stereo')
reader.example_description

In [ ]:
for x in reader.example_identifiers[0:10]:
    print(x)
#(filename,event)

In [ ]:
NUM_IMAGES_TO_PLOT = 2
i = 0
while i < NUM_IMAGES_TO_PLOT:
    example = reader[i]
    image = example[0]  
    plt.figure()
    if reader.mode == 'stereo':
        plt.title('event number : {}'.format(i))
        plt.pcolor(image[0,:,:,0],cmap='viridis') #stereo 
        plt.axes().set_aspect('equal')
        plt.show()
        plt.close()
        plt.pcolor(image[1,:,:,0],cmap='viridis')
        plt.axes().set_aspect('equal')
        plt.show()
        plt.close()
    elif reader.mode == 'mono':
        plt.title('image number : {}'.format(i))
        plt.pcolor(image[:,:,0],cmap='viridis') #mono charge
        plt.axes().set_aspect('equal')
        plt.title('charge')
        plt.show()
        plt.close()
    
    i+=1

# reader with yml 

https://github.com/ctlearn-project/ctlearn/blob/multitask/config/example_config.yml #TODO update to new version 

Input data section of yml file of parameters_example.yml

Data:

    file_list: /path/to/your/file
    mode: 'stereo' #mono
    image_selection_from_file:
    -
        name: 'leakage_parameters_filter'  #we have hillas, leakage, concentration, timing and morphology 
        args: {algorithm: 0, parameter: 'leakage_intensity_2', threshold_min: 0.0, threshold_max: 0.2}
    -
        name: 'hillas_parameters_filter'  #we have hillas, leakage, concentration, timing and morphology 
        args: {algorithm: 0, parameter: 'hillas_intensity', threshold_min: 1000.0}
        
    selected_telescope_type: 'LST_LST_LSTCam'
    selected_telescope_ids:
        'LST_LSTCam': [1]
    shuffle: true
    seed: 1111
    training_parameters: ['hillas_intensity','leakage_intensity_2']   
    
    image_channels: ['charge', 'peak_time', 'image_mask0']
    mapping_settings:
        camera_types: ['LSTCam']
        mapping_method:
            'LSTCam': 'bilinear_interpolation'
        padding:
            'LSTCam': 2
        mask_interpolation: false               
    
    event_info:
        - 'mc_energy'
    transforms:
        - name: 'MCEnergy'

In [ ]:
yml = '/mnt/lromanato/anaconda3/envs/LST37/CTA/yml_files/parameters_example.yml'
with open(yml, 'r') as config_file:
            config = yaml.safe_load(config_file)
config['Data'] = setup_DL1DataReader(config,'train')

In [ ]:
reader = DL1DataReader(**config['Data'])
print('mode: ',reader.mode)
print("Image shape: {}".format(reader[0][0].shape))
print("Example Description: ", reader.example_description)

In [ ]:
for x in reader.example_identifiers[0:4]:
    print(x)
#(filename,nrow,image_index,telescope_id, [selected parameters in yml file])

In [ ]:
NUM_IMAGES_TO_PLOT = 2
i = 0
while i < NUM_IMAGES_TO_PLOT:
    example = reader[i]
    image = example[0]  
    plt.figure()
    if reader.mode == 'stereo':
        plt.pcolor(image[0,:,:,0],cmap='viridis') #stereo 
        plt.axes().set_aspect('equal')
        plt.show()
        plt.close()
        plt.pcolor(image[1,:,:,0],cmap='viridis')
        plt.axes().set_aspect('equal')
        plt.show()
        plt.close()
    elif reader.mode == 'mono':
        print('image parameters of image number {} : {}'.format(i,reader.example_identifiers[i][4]))
        plt.figure(figsize=(12, 3))
        plt.subplots_adjust(wspace=0.31)
        plt.subplot(1, 3, 1)
        plt.pcolor(image[:,:,0],cmap='viridis')
        plt.title("PhotEv image number %i" %i)

        plt.subplot(1, 3, 2)
        plt.pcolor(image[:,:,1],cmap='viridis')
        plt.title("Timing image number %i" %i)

        plt.subplot(1, 3, 3)
        plt.pcolor(image[:,:,2],cmap='viridis')
        plt.title("Mask 0 image number %i" %i)

        plt.show()
        plt.close()
    
    i+=1
